## TEST 4 (WORKS but!)
- if you change the url, it will replace the first folder
- it will create folder "images" then put everything inside with numbered folder
- it's good if we want to scrape current chapter until end (it will some point stopped by server because limited access though)

In [ ]:
import httpx
from selectolax.parser import HTMLParser
import os
import random
import time
from dataclasses import dataclass
from urllib.parse import urljoin

USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0"
BASE_URL = "https://www.mgeko.cc"

@dataclass
class Response:
    body_html: HTMLParser
    next_page: dict

def create_image_directory(image_dir: str) -> None:
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)

def download_image(img_url: str, image_dir: str, page_number: int) -> None:
    headers = {"User-Agent": USER_AGENT}
    try:
        img_response = httpx.get(img_url, headers=headers)
        img_response.raise_for_status()
        img_alt = img_url.split("/")[-1].split(".")[0]
        page_dir = os.path.join(image_dir, f"page{page_number}")
        create_image_directory(page_dir)
        filename = f"{img_alt}.jpg"
        with open(os.path.join(page_dir, filename), "wb") as f:
            f.write(img_response.content)
            print(f"Image '{filename}' downloaded successfully to page{page_number} directory.")
    except httpx.RequestError as e:
        print(f"Failed to download image '{img_alt}.jpg': {e}")

def get_img(client: httpx.Client, url: str, image_dir: str, page_number: int) -> None:
    headers = {"User-Agent": USER_AGENT}
    try:
        resp = client.get(url, headers=headers)
        resp.raise_for_status()
        html = HTMLParser(resp.text)
        imgs = html.css("#chapter-reader img")
        img_urls = [i.attrs["src"] for i in imgs]
        for img_url in img_urls:
            download_image(img_url, image_dir, page_number)
    except httpx.RequestError as e:
        print(f"Failed to fetch images: {e}")

def get_next_page_url(html: HTMLParser) -> dict:
    next_page = html.css_first("#save > a.nextchap")
    if next_page:
        return {"href": next_page.attributes["href"]}
    return {"href": False}

def get_pages(client: httpx.Client, url: str, image_dir: str, base_url: str, page_number: int = 1) -> None:
    try:
        response = client.get(url, headers={"User-Agent": USER_AGENT})
        response.raise_for_status()
        html = HTMLParser(response.text)
        get_img(client, url, image_dir, page_number)
        next_page_url = get_next_page_url(html)
        if next_page_url["href"]:
            next_page_url = urljoin(base_url, next_page_url["href"])
            print(f"Next page: {next_page_url}")
            get_pages(client, next_page_url, image_dir, base_url, page_number + 1)
    except httpx.RequestError as e:
        print(f"Failed to fetch page: {e}")

def main() -> None:
    url = "https://www.mgeko.cc/reader/en/stuck-in-the-tower-chapter-61-eng-li/"
    image_dir = "images"
    create_image_directory(image_dir)
    client = httpx.Client()
    get_pages(client, url, image_dir, BASE_URL)

if __name__ == "__main__":
    main()

## TEST 5
- it doesn't exist it will create new folder on D:/manga using the name based on url
- if it exist, then do nothing
- it creates new subfolder for each chapter according to chapter number in url, WHETHER it exist or not (meaning replace by new file)
- it lacks checking existing file instead replaces with a new one. It will consume data by redownload file. 

In [ ]:
import httpx
from selectolax.parser import HTMLParser
import os
import random
import time
from dataclasses import dataclass
from urllib.parse import urljoin

USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0"
BASE_URL = "https://www.mgeko.cc"

@dataclass
class Response:
    body_html: HTMLParser
    next_page: dict

# Will create new manga folder if doesn't exist 
# Will rename folder based on url
def create_image_directory(image_dir: str) -> None:
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)

# Will turn the list of image links (from get_img()) to a folder name corresponding to their chapter 
# Will rename folder chapter using url links (url) from def main()
# Will Rename jpg file using url links (img_url) from def get_img()
# Will save each jpg into their respective folder (image_dir)
def download_image(img_url: str, image_dir: str, url: str, page_number: int) -> None:
    headers = {"User-Agent": USER_AGENT}
    try:
        img_response = httpx.get(img_url, headers=headers)
        img_response.raise_for_status()
        img_alt = img_url.split("/")[-1].split(".")[0]
        chapter_number = url.split("/")[-2].split("-chapter-")[1].split("-")[0]
        page_dir = os.path.join(image_dir, f"Chapter {chapter_number}")
        create_image_directory(page_dir)
        filename = f"{img_alt}.jpg"
        with open(os.path.join(page_dir, filename), "wb") as f:
            f.write(img_response.content)
            print(f"Image '{filename}' downloaded successfully to Chapter {chapter_number} directory.")
    except httpx.RequestError as e:
        print(f"Failed to download image '{img_alt}.jpg': {e}")

def get_img(client: httpx.Client, url: str, image_dir: str, page_number: int) -> None:
    headers = {"User-Agent": USER_AGENT}
    try:
        resp = client.get(url, headers=headers)
        resp.raise_for_status()
        html = HTMLParser(resp.text)
        imgs = html.css("#chapter-reader img")
        img_urls = [i.attrs["src"] for i in imgs]
        for img_url in img_urls:
            download_image(img_url, image_dir, url, page_number)
    except httpx.RequestError as e:
        print(f"Failed to fetch images: {e}")

def get_next_page_url(html: HTMLParser) -> dict:
    next_page = html.css_first("#save > a.nextchap")
    if next_page:
        return {"href": next_page.attributes["href"]}
    return {"href": False}

def get_pages(client: httpx.Client, url: str, image_dir: str, base_url: str, page_number: int = 1) -> None:
    try:
        response = client.get(url, headers={"User-Agent": USER_AGENT})
        response.raise_for_status()
        html = HTMLParser(response.text)
        get_img(client, url, image_dir, page_number)
        next_page_url = get_next_page_url(html)
        if next_page_url["href"]:
            next_page_url = urljoin(base_url, next_page_url["href"])
            print(f"Next page: {next_page_url}")
            get_pages(client, next_page_url, image_dir, base_url, page_number + 1)
    except httpx.RequestError as e:
        print(f"Failed to fetch page: {e}")

def main() -> None:
    url = "https://www.mgeko.cc/reader/en/against-the-gods-reuploaded-chapter-2-eng-li/"
    # Current Working Directory
    os.chdir("D:/manga")
    # Folder will be rename according to url
    chapter_name = url.split("/")[-2].split("-chapter-")[0].replace("-", " ")
    image_dir = chapter_name
    # from previous define function to create main folder if doesn't exist
    create_image_directory(image_dir)
    client = httpx.Client()
    get_pages(client, url, image_dir, BASE_URL)

if __name__ == "__main__":
    main()

### THIS IS HOW THE FOLDER RENAME WORKS

In [74]:
url = "https://www.mgeko.cc/reader/en/against-the-gods-reuploaded-chapter-44-eng-li/"
judul = url.split("/")
print(judul[5])
print(judul[-2])
print(judul[-2].split("-chapter-")[0])
print(judul[-2].split("-chapter-")[0].replace("-", " "))



against-the-gods-reuploaded-chapter-44-eng-li
against-the-gods-reuploaded-chapter-44-eng-li
against-the-gods-reuploaded
against the gods reuploaded


In [87]:
url = "https://www.mgeko.cc/reader/en/against-the-gods-reuploaded-chapter-44-eng-li/"
chapter = url.split("/")
print(chapter[5])
print(chapter[5].split("-chapter")[1])
print(chapter[5].split("-chapter-")[1].split("-")[0])


against-the-gods-reuploaded-chapter-44-eng-li
-44-eng-li
44


                                      +---------------+
                                      |  main()      |
                                      +---------------+
                                             |
                                             |
                                             v
                                      +---------------+
                                      |  Set URL,     |
                                      |  image_dir,   |
                                      |  and BASE_URL  |
                                      +---------------+
                                             |
                                             |
                                             v
                                      +---------------+
                                      |  create_image_directory  |
                                      |  (create main folder)    |
                                      +---------------+
                                             |
                                             |
                                             v
                                      +---------------+
                                      |  get_pages()    |
                                      |  (recursive function)  |
                                      +---------------+
                                             |
                                             |
                                             v
                                      +---------------+
                                      |  get_img()     |
                                      |  (fetch images)    |
                                      +---------------+
                                             |
                                             |
                                             v
                                      +---------------+
                                      |  download_image  |
                                      |  (download image)  |
                                      +---------------+
                                             |
                                             |
                                             v
                                      +---------------+
                                      |  get_next_page_url  |
                                      |  (get next page URL)  |
                                      +---------------+
                                             |
                                             |
                                             v
                                      +---------------+
                                      |  get_pages()    |
                                      |  (recursive call)  |
                                      +---------------+

## TEST DEBUGGING

In [ ]:
import httpx
from selectolax.parser import HTMLParser
import os
import random
import time
from dataclasses import dataclass
from urllib.parse import urljoin

USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0"
BASE_URL = "https://www.mgeko.cc"

@dataclass
class Response:
    body_html: HTMLParser
    next_page: dict

# Will create new manga folder if doesn't exist 
# Will rename folder based on url
def create_image_directory(image_dir: str) -> None:
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)

# Will turn the list of image links (from get_img()) to a folder name corresponding to their chapter 
# Will rename folder chapter using url links (url) from def main()
# Will Rename jpg file using url links (img_url) from def get_img()
# Will save each jpg into their respective folder (image_dir)
def download_image(img_url: str, image_dir: str, url: str, page_number: int) -> None:
    headers = {"User-Agent": USER_AGENT}
    try:
        img_response = httpx.get(img_url, headers=headers)
        img_response.raise_for_status()
        img_alt = img_url.split("/")[-1].split(".")[0]
        chapter_number = url.split("/")[-2].split("-chapter-")[1].split("-")[0]
        page_dir = os.path.join(image_dir, f"Chapter {chapter_number}")
        create_image_directory(page_dir)
        filename = f"{img_alt}.jpg"
        with open(os.path.join(page_dir, filename), "wb") as f:
            f.write(img_response.content)
            print(f"Image '{filename}' downloaded successfully to Chapter {chapter_number} directory.")
    except httpx.RequestError as e:
        print(f"Failed to download image '{img_alt}.jpg': {e}")

def get_img(client: httpx.Client, url: str, image_dir: str, page_number: int) -> None:
    headers = {"User-Agent": USER_AGENT}
    try:
        resp = client.get(url, headers=headers)
        resp.raise_for_status()
        html = HTMLParser(resp.text)
        imgs = html.css("#chapter-reader img")
        img_urls = [i.attrs["src"] for i in imgs]
        for img_url in img_urls:
            download_image(img_url, image_dir, url, page_number)
    except httpx.RequestError as e:
        print(f"Failed to fetch images: {e}")

def get_next_page_url(html: HTMLParser) -> dict:
    next_page = html.css_first("#save > a.nextchap")
    if next_page:
        return {"href": next_page.attributes["href"]}
    return {"href": False}

def get_pages(client: httpx.Client, url: str, image_dir: str, base_url: str, page_number: int = 1) -> None:
    try:
        response = client.get(url, headers={"User-Agent": USER_AGENT})
        response.raise_for_status()
        html = HTMLParser(response.text)
        get_img(client, url, image_dir, page_number)
        next_page_url = get_next_page_url(html)
        if next_page_url["href"]:
            next_page_url = urljoin(base_url, next_page_url["href"])
            print(f"Next page: {next_page_url}")
            get_pages(client, next_page_url, image_dir, base_url, page_number + 1)
    except httpx.RequestError as e:
        print(f"Failed to fetch page: {e}")

def main() -> None:
    url = "https://www.mgeko.cc/reader/en/against-the-gods-reuploaded-chapter-2-eng-li/"
    # Current Working Directory
    os.chdir("D:/manga")
    # Folder will be rename according to url
    chapter_name = url.split("/")[-2].split("-chapter-")[0].replace("-", " ")
    image_dir = chapter_name
    # from previous define function to create main folder if doesn't exist
    create_image_directory(image_dir)
    client = httpx.Client()
    get_pages(client, url, image_dir, BASE_URL)

if __name__ == "__main__":
    main()